In [1]:
from helper import create_batches
import tensorflow as tf
import numpy as np
import shelve
import joblib
import time
import math
import random

#Cuz the file is inside 'code' directory
mount_point = "../shelved_data/"

with shelve.open(mount_point+'IAM_Data') as shelf:
    vocabulary = shelf['chars']
    list_of_images = shelf['list_of_images']
    image_labels = shelf['image_labels']
    
image_arrays = joblib.load(mount_point+'image_arrays')

#List_images ko sort karo
list_of_images.sort()

#Convert vocabulary to list
vocabulary = list(vocabulary)
#Sort so as to have the same ordering every time..
vocabulary.sort()
vocabulary.append("<Blank>")

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
#Model parameters
img_height = 104
img_width = 688
vocab_size = len(vocabulary)

#Common Hyper Parameters
epochs = 700
batch_size = 250 #Should be proportional to the number of Images

#LSTM Params
rnn_hidden_units = 200

conv_out_height, conv_out_width = (int(math.ceil(img_height/(2**3 * 3))),int(math.ceil(img_width/(2**3 * 3))))

#Number of time_steps to unroll for..
seq_len = conv_out_height * conv_out_width

## Save my MoDel

In [3]:
#Take first 12000 images...
training_list = list_of_images[:12000]
testing_list = list_of_images[12000:12250]
random.shuffle(training_list)

In [4]:
train_batches_x,train_batches_y = create_batches(batch_size,training_list,image_arrays,image_labels,vocabulary)
test_batches_x,test_batches_y = create_batches(len(testing_list),testing_list,image_arrays,image_labels,vocabulary)

## Predict using Model

In [5]:
resume_epoch = 620

In [ ]:
with tf.Session() as sess:
    # Load the graph
    saver = tf.train.import_meta_graph('../model/Lines_RNN_'+str(resume_epoch)+'.meta')

    # Restore the weights and biases
    saver.restore(sess, '../model/Lines_RNN_'+str(resume_epoch))

    inputs = sess.graph.get_tensor_by_name('Placeholder:0')
    
    target_indices = sess.graph.get_tensor_by_name('targets/indices:0')
    target_values = sess.graph.get_tensor_by_name('targets/values:0')
    target_shape = sess.graph.get_tensor_by_name('targets/shape:0')

    is_test = sess.graph.get_tensor_by_name('Placeholder_1:0')
    iteration = sess.graph.get_tensor_by_name('Placeholder_2:0')
    time_steps = sess.graph.get_tensor_by_name('Placeholder_3:0')
    dropout_lstm = sess.graph.get_tensor_by_name('Placeholder_4:0')
    dropout_fc = sess.graph.get_tensor_by_name('Placeholder_5:0')
    
        
    train = sess.graph.get_operation_by_name('RMSProp')
    update_ema = sess.graph.get_operation_by_name('group_deps')
    cost = sess.graph.get_tensor_by_name('Mean:0')
    label_error_rate = sess.graph.get_tensor_by_name('Mean_1:0')
    
    #For debugging purposes...
    fc_outputs_2_bn = sess.graph.get_tensor_by_name('batchnorm_6/add_1:0')

    #checkpoint flag
    checkpoint = False
    
    timer = 0
    
    for e in range(resume_epoch,epochs): 

        start_time = time.time()
    
        #Checkpoint every 1 epochs
        if (e%2)==0:
            checkpoint = True

        random.shuffle(training_list)
        train_batches_x,train_batches_y = create_batches(batch_size,training_list,image_arrays,image_labels,vocabulary)

        #Iterate through all images in a single epoch...
        for b in range(len(train_batches_x)):
            
            feed_train = {
                    inputs:train_batches_x[b].transpose([2,0,1]),
                    target_indices:train_batches_y[b][0],target_values:train_batches_y[b][1],target_shape:train_batches_y[b][2],
                    time_steps:np.array([seq_len]*batch_size),
                    is_test:False,
                    dropout_fc:np.array(0.7),dropout_lstm:np.array(0.8)
                   }

            feed_uma = {
                    inputs:train_batches_x[b].transpose([2,0,1]),
                    target_indices:train_batches_y[b][0],target_values:train_batches_y[b][1],target_shape:train_batches_y[b][2],
                    time_steps:np.array([seq_len]*batch_size),
                    is_test:False,iteration:len(train_batches_x)*e + b,
                    dropout_fc:np.array(1.0),dropout_lstm:np.array(1.0)
                   }

        
            _,debug_out = sess.run([train,fc_outputs_2_bn],feed_dict=feed_train)
            sess.run(update_ema,feed_dict=feed_uma)
            

        if checkpoint:
            last_cost,train_ler = sess.run([cost,label_error_rate],feed_dict=feed_train)

        #After iterating through all batches..
        test_batch_size = len(testing_list)
      
        feed_test = {
            inputs:test_batches_x[0].transpose([2,0,1]),
            target_indices:test_batches_y[0][0],target_values:test_batches_y[0][1],target_shape:test_batches_y[0][2],
            time_steps:np.array([seq_len]*test_batch_size),
            is_test:True,
            dropout_fc:np.array(1.0),dropout_lstm:np.array(1.0)
           }
            

        #Evaluate the model, and store every 5 epochs...
        if checkpoint:

            #Accuracy on test_data
            ler_val = sess.run(label_error_rate,feed_dict=feed_test)                

            end_time = time.time()       
            time_taken = end_time - start_time
            timer += time_taken

            np.savez_compressed(str(e)+"_fcouts2bn",a=debug_out)

            print("{},{:.6f},{:.2f},{:.2f},{}\n".format(e,last_cost,train_ler,ler_val,timer))
#             np.savetxt(str(e)+"_fcouts2bn.txt",debug_out,delimiter=',')
#             with open('debug.txt','a') as d:
#                 d.write(debug_out)
                
            with open('progress.csv','a') as f:
                f.write("{},{:.6f},{:.2f},{:.2f},{}\n".format(e,last_cost,train_ler,ler_val,timer))

#             Save the model
            saver.save(sess,'../model/Lines_RNN_'+str(e))

            checkpoint = False
            timer = 0
        
        else:
            end_time = time.time()       
            time_taken = end_time - start_time
            timer += time_taken

INFO:tensorflow:Restoring parameters from ../model/Lines_RNN_620
620,102.244720,0.59,0.84,152.1252200603485

INFO:tensorflow:../model/Lines_RNN_620 is not in all_model_checkpoint_paths. Manually adding it.
622,101.425079,0.63,0.91,153.6421046257019

INFO:tensorflow:../model/Lines_RNN_622 is not in all_model_checkpoint_paths. Manually adding it.
624,102.806831,0.66,0.86,151.15894079208374

INFO:tensorflow:../model/Lines_RNN_624 is not in all_model_checkpoint_paths. Manually adding it.
626,104.563675,0.70,0.84,149.40807247161865

INFO:tensorflow:../model/Lines_RNN_626 is not in all_model_checkpoint_paths. Manually adding it.
628,101.582375,0.57,0.86,149.62976551055908

INFO:tensorflow:../model/Lines_RNN_628 is not in all_model_checkpoint_paths. Manually adding it.
630,102.903816,0.60,0.90,149.43674278259277

INFO:tensorflow:../model/Lines_RNN_630 is not in all_model_checkpoint_paths. Manually adding it.
632,100.351097,0.64,0.83,149.3182294368744

INFO:tensorflow:../model/Lines_RNN_632 is